In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

## Hovering study

In [ ]:
exp_name = "2021_07_27_hover"; 

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
results_df = results_df.iloc[1:]

In [ ]:
from calibration import get_calibration_function_median
calib_function_median, freqs = get_calibration_function_median(
    "2021_07_08_stepper_fast", mic_type="audio_deck", snr=5, motors="all45000"
)
plt.figure()
for i in range(4):
    plt.plot(freqs, calib_function_median(freqs)[i])

In [ ]:
from matplotlib import cm
cmap = cm.get_cmap('inferno')

min_z = 0.4

fig_pos, ax_pos = plt.subplots()
fig_pos.set_size_inches(3, 3)
ax_pos.set_xlabel('x [cm]')
ax_pos.set_ylabel('y [cm]')
ax_pos.axis('equal')
ax_pos.grid()

for i_row, row in results_df.iterrows():
    print(row.appendix)

    valid = row.positions[:, 2] > min_z
    stft = row.stft[valid, ...]
    frequencies_matrix = row.frequencies_matrix[valid, ...]

    START_Y = 0 #row.positions[0, 1]
    START_X = row.positions[0, 0]
    #print("starting at", START_X, START_Y)

    if row.appendix in ["", "_30"]:
        START_Y -= 0.3 # m 
    else:
        START_Y -= 0.5 # m 
    positions_cm = np.c_[row.positions[valid, 0] - START_X,
                         row.positions[valid, 1] - START_Y] * 1e2

    #plt.figure()
    #plt.title(row.appendix)
    label = f'appendix: "{row.appendix}"'
    ax_pos.scatter(positions_cm[:, 0], positions_cm[:, 1], label=label)
    
    fig = plt.figure()
    fig.set_size_inches(6, 3)
    total = 0
    for mic_i in range(stft.shape[1]):
        for i in range(stft.shape[0]):
            amps = np.abs(stft[i, mic_i, :])
            freqs = frequencies_matrix[i]
            total += np.sum(amps ** 2)
            plt.plot(freqs[freqs > 0], amps[freqs > 0], color=cmap(i/stft.shape[0]),alpha=0.2)
        plt.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
                 color='C0')
    #print("total energy", total)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('amplitude')
    plt.title(label)
    plt.ylim(0, 20)
    plt.grid()
    
    #fig = plt.figure()
    #fig.set_size_inches(6, 3)
    #plt.ylim(0, 20)
    #plt.grid()
    #plt.xlabel('frequency [Hz]')
    #plt.ylabel('amplitude')

In [ ]:
exp_name = "2021_07_08_stepper_fast"; 

results_compare = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")

delta = 5

distance 

for distance in [30, 49]:
    rows = results_df.loc[(results_compare.distance >= distance - delta) & 
                          (results_compare.distance <= distance + delta) & 
                          (results_compare.bin_selection == 5) & 
                          (results_compare.motors=="all45000"), :]
    
    fig = plt.figure()
    fig.set_size_inches(6, 3)
    for i_row, (_, row) in enumerate(rows.iterrows()):
        stft = row.stft
        frequencies_matrix = row.frequencies_matrix
        
        for mic_i in range(stft.shape[1]):
            for i in range(stft.shape[0]):
                amps = np.abs(stft[i, mic_i, :])
                freqs = frequencies_matrix[i]
                plt.plot(freqs[freqs > 0], amps[freqs > 0], color=cmap(i_row/len(rows)),
                         alpha=0.2)
                
            plt.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
                     color='C0')
    #print("total energy", total)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('amplitude')
    plt.title(f"distances {rows.distance.min()} to {rows.distance.max()}")
    plt.ylim(0, 20)
    plt.xlim(3000, 4500)
    plt.grid()

In [ ]:
from inference import Inference
from crazyflie_description_py.experiments import WALL_ANGLE_DEG
from estimators import DistanceEstimator

from scipy.signal import find_peaks

inf_machine = Inference()
dist_range = [10, 70]
inf_machine.add_geometry(dist_range, WALL_ANGLE_DEG)
inf_machine.add_calibration_function(calib_function_median)

mics = [0]


for i_row, row in results_df.iterrows():
    valid = row.positions[:, 2] > min_z
    stft = row.stft[valid, ...]
    frequencies_matrix = row.frequencies_matrix[valid, ...]
    
    n_steps = stft.shape[0]
    
    total = None
    
    fig_all, ax_all = plt.subplots()
    d_estimates = []
    for i in range(n_steps):
        freqs = frequencies_matrix[i]
        slice_f = np.abs(stft[i]) # n_mics x n_buffer
        
        inf_machine.add_data(slice_f[:, freqs>0], freqs[freqs>0])
        #inf_machine.filter_out_freqs(freq_ranges=[[3550,3770]])
        
        #fig, ax = plt.subplots()
        distance_estimator = DistanceEstimator()
        for i_mic in mics:
            dists, prob, diff = inf_machine.do_inference(algorithm="bayes", mic_idx=i_mic)
            
            #inf_machine.plot(i_mic, ax=ax)
            
            distance_estimator.add_distribution(diff*1e-2, prob, i_mic)
        
        dists_all, prob_all = distance_estimator.get_distance_distribution(
            method='sum', 
            distances_m = np.arange(min(dist_range), max(dist_range))  * 1e-2
        )
        peaks, *_ = find_peaks(prob_all)
        d_estimate = list(dists_all[peaks[:2]] * 1e2)
        
        if total is None:
            total = prob_all 
        else:
            total *= prob_all
            
        ax_all.plot(dists_all*1e2, prob_all, color=cmap(i/n_steps))
        [ax_all.axvline(d, color=cmap(i/n_steps), ls=":") for d in d_estimate]
        ax_all.set_title(f"appendix: {row.appendix}")
        
        d_estimates += d_estimate
        
        
        if i > n_steps: break
            
    print('distance estimates', d_estimates)
            
    total /= np.sum(total)
    ax_all.plot(dists_all*1e2, total, color="k", ls=":")
    
    plt.figure()
    plt.plot(dists_all*1e2, total)
    plt.title('sum of all probs')
    #plt.yscale("log")
    #plt.ylim(1e-3, 1)

In [ ]:
## compare to stepper motor variance 

## Manual study